In [1]:
# !mamba install -c conda-forge geopandas
# !mamba install -c conda-forge geopandas fiona gdal pyogrio

In [3]:
from helpers.locations import *
from helpers.drugs import *
from helpers.dates import *
from helpers.population import CountyNormalizer
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import numpy as np

In [23]:
state = "Colorado"
legalisation_date = "2014-01-01"

county_normalizer = CountyNormalizer()

# df_drug = pd.read_csv("data/drugs/US_Oxycodone.csv")
# DRUG_NAMES = ["US_Amphetamine", "US_Codeine", "US_Hydrocodone", "US_Morphine", "US_Oxycodone"]

# Use glob to find files matching the pattern
DRUG_DATA_PATH = "data/drugs/"
drug_files = glob.glob(os.path.join(DRUG_DATA_PATH, "US_*.csv"))

# Extract the base names without the file extension
DRUG_NAMES = [os.path.splitext(os.path.basename(file))[0] for file in drug_files]
# SPARSE_NAMES = ["US_Tincured", "US_IMethamphetamine", "US_THC"]
# for sparse in SPARSE_NAMES:
#     DRUG_NAMES.remove(sparse)

# DRUG_NAMES = DRUG_NAMES[:10]

df_drug = get_drugs_dfs(DRUG_NAMES)
df_drug = df_drug.generate_state_column("placeDcid")
df_drug = df_drug.generate_year()
df_drug['FIPS'] = df_drug['placeDcid'].str.split('/').str[-1]
df_drug["population"] = df_drug.apply(county_normalizer.get_population, axis=1)
# df_drug.dropna(inplace= True)
df_drug.dropna(subset=["population"], inplace=True)
df_drug

,placeDcid,placeName,Date,US_IMethamphetamine,US_Morphine,US_Amphetamine,US_THC,US_Ketmaine,US_Methadone,US_Fentanyl,...,US_Cocaine,US_Methamphetamine,US_Codeine,US_Tincured,US_Hydrocodone,US_Oxycodone,state,Year,FIPS,population
0,geoId/01001,Autauga County,2010-01,0.00,731.49,610.62,NaN,NaN,300.78,22.63,...,0.14,NaN,402.52,NaN,2566.96,1160.68,Alabama,2010,01001,54761.0
1,geoId/01001,Autauga County,2010-04,0.00,740.60,617.29,NaN,NaN,299.61,21.50,...,0.56,NaN,429.87,NaN,2741.98,1174.64,Alabama,2010,01001,54761.0
2,geoId/01001,Autauga County,2010-07,0.00,690.43,583.80,NaN,NaN,279.13,20.62,...,0.26,NaN,393.40,NaN,2744.03,1125.83,Alabama,2010,01001,54761.0
3,geoId/01001,Autauga County,2010-10,0.15,705.21,687.77,NaN,NaN,306.29,20.49,...,0.28,NaN,460.70,NaN,2920.35,1259.38,Alabama,2010,01001,54761.0
4,geoId/01001,Autauga County,2011-01,0.07,731.07,726.76,NaN,NaN,286.19,19.38,...,0.46,NaN,541.20,NaN,3070.24,1156.38,Alabama,2011,01001,55229.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150763,geoId/56045,NaN,2016-10,NaN,42.03,59.48,NaN,NaN,5.48,0.75,...,0.10,NaN,45.19,NaN,103.31,159.98,Wyoming,2016,56045,7228.0
150764,geoId/56045,NaN,2017-01,NaN,43.06,58.34,NaN,NaN,5.37,0.73,...,0.07,NaN,46.35,NaN,100.78,153.44,Wyoming,2017,56045,6962.0
150765,geoId/56045,NaN,2017-04,NaN,52.34,54.30,NaN,NaN,4.84,0.54,...,0.03,NaN,32.44,NaN,95.51,154.04,Wyoming,2017,56045,6962.0
150766,geoId/56045,NaN,2017-07,NaN,37.70,62.98,NaN,NaN,4.89,0.83,...,0.14,NaN,36.01,NaN,91.86,142.69,Wyoming,2017,56045,6962.0


In [24]:
# Loop through each drug column and divide by 'population' column
for drug in DRUG_NAMES:
    df_drug[drug] = df_drug[drug] / df_drug['population']
df_drug

,placeDcid,placeName,Date,US_IMethamphetamine,US_Morphine,US_Amphetamine,US_THC,US_Ketmaine,US_Methadone,US_Fentanyl,...,US_Cocaine,US_Methamphetamine,US_Codeine,US_Tincured,US_Hydrocodone,US_Oxycodone,state,Year,FIPS,population
0,geoId/01001,Autauga County,2010-01,0.000000,0.013358,0.011151,NaN,NaN,0.005493,0.000413,...,0.000003,NaN,0.007350,NaN,0.046876,0.021195,Alabama,2010,01001,54761.0
1,geoId/01001,Autauga County,2010-04,0.000000,0.013524,0.011272,NaN,NaN,0.005471,0.000393,...,0.000010,NaN,0.007850,NaN,0.050072,0.021450,Alabama,2010,01001,54761.0
2,geoId/01001,Autauga County,2010-07,0.000000,0.012608,0.010661,NaN,NaN,0.005097,0.000377,...,0.000005,NaN,0.007184,NaN,0.050109,0.020559,Alabama,2010,01001,54761.0
3,geoId/01001,Autauga County,2010-10,0.000003,0.012878,0.012559,NaN,NaN,0.005593,0.000374,...,0.000005,NaN,0.008413,NaN,0.053329,0.022998,Alabama,2010,01001,54761.0
4,geoId/01001,Autauga County,2011-01,0.000001,0.013237,0.013159,NaN,NaN,0.005182,0.000351,...,0.000008,NaN,0.009799,NaN,0.055591,0.020938,Alabama,2011,01001,55229.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150763,geoId/56045,NaN,2016-10,NaN,0.005815,0.008229,NaN,NaN,0.000758,0.000104,...,0.000014,NaN,0.006252,NaN,0.014293,0.022133,Wyoming,2016,56045,7228.0
150764,geoId/56045,NaN,2017-01,NaN,0.006185,0.008380,NaN,NaN,0.000771,0.000105,...,0.000010,NaN,0.006658,NaN,0.014476,0.022040,Wyoming,2017,56045,6962.0
150765,geoId/56045,NaN,2017-04,NaN,0.007518,0.007799,NaN,NaN,0.000695,0.000078,...,0.000004,NaN,0.004660,NaN,0.013719,0.022126,Wyoming,2017,56045,6962.0
150766,geoId/56045,NaN,2017-07,NaN,0.005415,0.009046,NaN,NaN,0.000702,0.000119,...,0.000020,NaN,0.005172,NaN,0.013194,0.020496,Wyoming,2017,56045,6962.0


In [25]:
# df_drug_state = df_drug[df_drug["state"] == state]
# df_drug_state

In [26]:
# # Example legalisation_date
# # legalisation_date_ts = pd.Timestamp("2014-01-01")  # Replace with actual date

# DATE_COLUMN = "Date"

# # Split into two data frames
# df_before_legalisation = df_drug_state[df_drug_state[DATE_COLUMN] < legalisation_date]
# df_after_legalisation = df_drug_state[df_drug_state[DATE_COLUMN] >= legalisation_date]
# df_before_legalisation

# Visualisations

In [27]:
# ORIGINAL_COL = "US_Codeine"
# GRAPH_COL = ORIGINAL_COL+"_Log"
# df_drug[GRAPH_COL] = np.log(df_drug[ORIGINAL_COL])

# # Create dataframe and process FIPS codes
# cropped_df = df_drug[df_drug["Date"] == "2017-10"]

# # Load US counties shapefile
# counties = gpd.read_file('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
# merged = counties.merge(cropped_df, how='left', left_on='id', right_on='FIPS')

# # Create the plot
# fig, ax = plt.subplots(1, figsize=(15, 10))

# # Create choropleth map
# merged.plot(
#     column=GRAPH_COL,
#     ax=ax,
#     legend=True,
#     legend_kwds={'label': f'{GRAPH_COL} Usage'},
#     missing_kwds={'color': 'lightgrey'},
#     cmap='YlOrRd'
# )

# # Set the map boundaries to focus on continental US
# ax.set_xlim([-125, -66.5])  # Longitude limits for continental US
# ax.set_ylim([24, 49])      # Latitude limits for continental US

# # Customize the map
# plt.title('Codeine Usage by County (2014)', pad=20)
# plt.axis('off')

# # Show the plot
# plt.show()

In [28]:
import os
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# SOURCE_NAME = "US_Oxycodone"

def generate_images(name):
    ORIGINAL_COL = name  # e.g., "US_Codeine"
    GRAPH_COL = ORIGINAL_COL + "_Log"
    
    # Apply log transformation
    df_drug[GRAPH_COL] = np.log(df_drug[ORIGINAL_COL].dropna() + 1e-10)  # Adding small constant to avoid log(0)

    # Create base output directory for maps
    base_output_dir = Path('drug_maps')
    base_output_dir.mkdir(exist_ok=True)
    
    # Create drug-specific directory
    drug_output_dir = base_output_dir / ORIGINAL_COL.lower()
    drug_output_dir.mkdir(exist_ok=True)

    # Get sorted list of unique dates
    dates = sorted(df_drug['Date'].unique())

    # Find the global min and max for consistent color scale
    global_min = df_drug[GRAPH_COL].min()
    global_max = df_drug[GRAPH_COL].max()

    # Load US counties shapefile (load once outside the loop for efficiency)
    counties = gpd.read_file('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')

    # Create a map for each date
    for date in dates:
        # Filter dataframe for this date
        cropped_df = df_drug[df_drug["Date"] == date]
        
        # Merge with geographic data
        merged = counties.merge(cropped_df, how='left', left_on='id', right_on='FIPS')
        
        # Create the plot
        fig, ax = plt.subplots(1, figsize=(15, 10))
        
        # Create choropleth map
        merged.plot(
            column=GRAPH_COL,
            ax=ax,
            legend=True,
            legend_kwds={'label': f'{GRAPH_COL} Usage'},
            missing_kwds={'color': 'lightgrey'},
            cmap='YlOrRd',
            vmin=global_min,  # Set consistent color scale
            vmax=global_max
        )
        
        # Set the map boundaries to focus on continental US
        ax.set_xlim([-125, -66.5])
        ax.set_ylim([24, 49])
        
        # Customize the map
        plt.title(f'{GRAPH_COL} Usage by County ({date})', pad=20)
        plt.axis('off')
        
        # Save the plot
        filename = drug_output_dir / f'{ORIGINAL_COL.lower()}_map_{date.replace("-", "_")}.png'
        plt.savefig(filename, bbox_inches='tight', dpi=300)
        plt.close()  # Close the figure to free memory

    print(f"Created maps for {len(dates)} dates in the '{drug_output_dir}' directory")

# Example usage
# generate_images(SOURCE_NAME)

In [33]:
def generate_gif(name: str):
    lower_name = name.lower()
    path = f"drug_maps/{lower_name}/{lower_name}_map_*.png"

    try:
        # Get list of saved images
        images = []
        files = sorted(glob.glob(path))
        
        if not files:
            print(f"No images found for {name}. Check the path: {path}")
            return

        # Load all images
        for filename in files:
            images.append(Image.open(filename))
        
        # Create output directory if it doesn't exist
        output_dir = Path(f"drug_maps/gifs")
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Save as GIF
        output_path = output_dir / f"{lower_name}_usage_animation.gif"
        images[0].save(
            output_path,
            save_all=True,
            append_images=images[1:],
            duration=500,  # Duration for each frame in milliseconds
            loop=0
        )
        print(f"Created animation: {output_path}")
    except ImportError:
        print("PIL (Pillow) library not installed. Skipping GIF creation.")
    except FileNotFoundError as e:
        print(f"File not found: {e}")

# Example usage
# generate_gif(SOURCE_NAME)

In [32]:
for source in DRUG_NAMES:
    print("generating images for", source, "...")
    generate_images(source)
    generate_gif(source)

generating images for  US_IMethamphetamine ...
Created maps for 48 dates in the 'drug_maps/us_imethamphetamine' directory
Created animation: drug_maps/us_imethamphetamine/us_imethamphetamine_usage_animation.gif
generating images for  US_Morphine ...
Created maps for 48 dates in the 'drug_maps/us_morphine' directory
Created animation: drug_maps/us_morphine/us_morphine_usage_animation.gif
generating images for  US_Amphetamine ...
Created maps for 48 dates in the 'drug_maps/us_amphetamine' directory
Created animation: drug_maps/us_amphetamine/us_amphetamine_usage_animation.gif
generating images for  US_THC ...
Created maps for 48 dates in the 'drug_maps/us_thc' directory
Created animation: drug_maps/us_thc/us_thc_usage_animation.gif
generating images for  US_Ketmaine ...
Created maps for 48 dates in the 'drug_maps/us_ketmaine' directory
Created animation: drug_maps/us_ketmaine/us_ketmaine_usage_animation.gif
generating images for  US_Methadone ...
Created maps for 48 dates in the 'drug_ma

In [34]:
for source in DRUG_NAMES:
    generate_gif(source)

Created animation: drug_maps/gifs/us_imethamphetamine_usage_animation.gif
Created animation: drug_maps/gifs/us_morphine_usage_animation.gif
Created animation: drug_maps/gifs/us_amphetamine_usage_animation.gif
Created animation: drug_maps/gifs/us_thc_usage_animation.gif
Created animation: drug_maps/gifs/us_ketmaine_usage_animation.gif
Created animation: drug_maps/gifs/us_methadone_usage_animation.gif
Created animation: drug_maps/gifs/us_fentanyl_usage_animation.gif
Created animation: drug_maps/gifs/us_powdered_opium_usage_animation.gif
Created animation: drug_maps/gifs/us_cocaine_usage_animation.gif
Created animation: drug_maps/gifs/us_methamphetamine_usage_animation.gif
Created animation: drug_maps/gifs/us_codeine_usage_animation.gif
Created animation: drug_maps/gifs/us_tincured_usage_animation.gif
Created animation: drug_maps/gifs/us_hydrocodone_usage_animation.gif
Created animation: drug_maps/gifs/us_oxycodone_usage_animation.gif
